In [3]:
dg_american = pd.read_csv(f"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market={market_type}&odds_format=american&file_format=csv&key={dg_key}")
dg_decimal = pd.read_csv(f"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market={market_type}&odds_format=decimal&file_format=csv&key={dg_key}")

# grab american and euro DataGolf odds for each player and combine
am_odds = dg_american[['player_name','datagolf_base_history_fit']].rename(columns={'datagolf_base_history_fit':'am_odds_dg'})
dec_odds = dg_decimal[['player_name','datagolf_base_history_fit']].rename(columns={'datagolf_base_history_fit':'dec_odds_dg'})
dg_odds = pd.merge(am_odds,dec_odds, on='player_name')

# get best sportbooks line for each player in american and euro formats
agg_am = dg_american.drop(columns=['player_name','dg_id','datagolf_baseline','datagolf_base_history_fit','last_updated','event_name','market']).T.max().to_frame()
agg_dec = dg_decimal.drop(columns=['player_name','dg_id','datagolf_baseline','datagolf_base_history_fit','last_updated','event_name','market']).T.max().to_frame()

df = pd.merge(dg_odds, agg_am, left_index=True, right_index=True)
df = pd.merge(df,agg_dec,left_index=True, right_index=True).T.drop_duplicates().T
df.columns = ['player_name', 'dg_american','dg_decimal','books_mean_american','books_mean_decimal']

df['market_type'] = market_type
df['market_target'] = df['market_type'].map(market_target_dict)
df['target_decimal'] = (df['dg_decimal'] * df['market_target']).astype(float)
df['target_american'] = df['target_decimal'].apply(lambda x: convert_euro_to_american(x))


# add expected value column (for color)
df['ev'] = ((1 / df['dg_decimal']) * df['books_mean_decimal'] -1)

In [2]:
dg_key = "e297e933c3ad47d71ec1626c299e"
market_type = 'top_5'

names_dict = {'Matt Fitzpatrick': 'Matthew Fitzpatrick',
    'Si Kim': 'Si Woo Kim',
    'Min Lee': 'Min Woo Lee',
    'Byeong An': 'Byeong Hun An',
    'Rooyen Van': 'Erik Van Rooyen',
    'Vince Whaley': 'Vincent Whaley',
    'Kevin Yu': 'Kevin Yu',
    'Kyounghoon Lee': 'Kyoung-Hoon Lee'
             }

def implied_probability(moneyline_odds):
    """
    Calculate the implied probability from moneyline odds.
    """
    # Convert negative odds to positive for calculation
    if moneyline_odds < 0:
        implied_prob = abs(moneyline_odds) / (abs(moneyline_odds) + 100)
    else:
        implied_prob = 100 / (moneyline_odds + 100)
    
    # Return implied probability as percentage
    return implied_prob

def fix_names(list_of_player_names):
    
    # swaps last/first into first/last name
    names = list_of_player_names.str.split(expand=True)                  
    names[0] = names[0].str.rstrip(",")
    names[1] = names[1].str.rstrip(",")
    names['player_name'] = names[0].str[0] + " " + names[1]

    # uses dictionary to correct known problem names
    # ie "Jr" or "Si Woo Kim")
    for incorrect_name, correct_name in names_dict.items():
        names['player_name'] = np.where(names['player_name'] == incorrect_name, correct_name, names['player_name'])

    return names.player_name

def plus_prefix(a):
    if a > 0:
        return f"+{a}"
    return a

market_target_dict = {
    'win':1.15,
    'top_5': 1.1,
    'top_10': 1.1,
    'top_20': 1.05
    }


def convert_euro_to_american(dec_odds):
    if dec_odds >= 2:
        return (dec_odds - 1) * 100
    else:
        return -100 / (dec_odds-1)
    
#     Function to generate CSS styles for background gradient based on column values
def background_gradient(val):
    # Assuming you want to map the gradient from blue to red based on the Color_Column values
    # You can adjust the colors and the gradient range as needed
    color = f'rgba(0, 0, {(val / df["ev"].max()) * 255}, 1)' 
    return f'background-color: {color}'

In [40]:
df = get_ev_table(market_type)

df = (
    df[['player_name','target_american','target_decimal','ev']].dropna()
    .assign(player_name=lambda x: fix_names(x['player_name']))
).rename(columns={'player_name':'Player','target_american':'Target US','target_decimal':'Target Euro','ev':'EV'})

df = df.dropna()
df = df.sort_values('Player')
# add styling  ("+" prefixes and color)
# df['Target US'] = df['Target US'].round(-2).astype(int).apply(lambda x: x if x <= 0 else f"+{x}")
# df['Target Euro'] = df['Target Euro'].astype(int).apply(lambda x: f"{x}-1")

In [4]:
import pandas as pd
import numpy as np
import streamlit as st

from utils import *

In [44]:
playcr = pd.read_excel(r"playcr_schwab.xlsx")

playcr = playcr[playcr.market=='top_5'].assign(player_name=lambda x: fix_names(x['player']))

playcr['player_name'] = playcr.player_name.str.title()

playcr = playcr[['player_name','market','line']].reset_index(drop=True)

In [45]:
playcr

,player_name,market,line
0,R Aaron,top_5,1000
1,S Adam,top_5,1400
2,S Adam,top_5,1000
3,S Adam,top_5,2000
4,B Akshay,top_5,1200
...,...,...,...
75,H Tom,top_5,1000
76,K Tom,top_5,1100
77,F Tony,top_5,650
78,P Victor,top_5,2000


In [42]:
playcr = playcr[['player_name','line']]

playcr

,player_name,line
0,R Aaron,1000
1,S Adam,1400
2,S Adam,1000
3,S Adam,2000
4,B Akshay,1200
...,...,...
75,H Tom,1000
76,K Tom,1100
77,F Tony,650
78,P Victor,2000


In [43]:
df = df[['Player','Target US']]

df.columns = ['player_name','dg_line']

df = df.reset_index(drop=True)

df

,player_name,dg_line
0,A Bhatia,1540.301069
1,A Eckroat,1404.256770
2,A Novak,4762.017737
3,A Putnam,1068.004514
4,A Rai,1366.231581
...,...,...
127,V Perez,6627.480167
128,W Bryan,23653.824006
129,W Simpson,3044.645764
130,Z Blair,8982.945775


In [36]:
# playcr = playcr.set_index('player_name')
# df = df.set_index('player_name')

pd.merge(playcr,df, how='left', on='player_name')

,player_name,line,dg_line
0,R Aaron,1000,NaN
1,S Adam,1400,NaN
2,S Adam,1000,NaN
3,S Adam,2000,NaN
4,B Akshay,1200,NaN
...,...,...,...
75,H Tom,1000,NaN
76,K Tom,1100,NaN
77,F Tony,650,NaN
78,P Victor,2000,NaN


In [48]:
df.player_name.isin(playcr.player_name).value_counts()

False    132
Name: player_name, dtype: int64